In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from keras.models import Model, Sequential
from keras.layers import Input, Dense
import networkx as nx

In [1]:
file_path = '/content/mapped_logs_2.csv'
log_data = pd.read_csv(file_path)

# Use a pretrained Sentence Transformer for semantic embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

log_data['embedding'] = list(model.encode(log_data['message'].fillna("").astype(str)))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings generated!


In [26]:
print(log_data['embedding'])

0       [0.015637953, 0.00022504911, 0.043485753, 0.01...
1       [-0.064811096, 0.047025338, -0.061534055, 0.02...
2       [0.0023149748, -0.004432573, -0.041655324, -0....
3       [-0.055262655, 0.037988104, -0.085704274, -0.0...
4       [-0.06708172, 0.049024645, -0.056578003, 0.024...
                              ...                        
2640    [0.016748793, -0.011854233, 0.047355477, -0.01...
2641    [0.028540818, 0.003513586, 0.025054125, -0.005...
2642    [-0.009375343, 0.071565405, 0.01157132, 0.0152...
2643    [-0.02759048, 0.0465453, -0.007963847, 0.03988...
2644    [-0.010922249, 0.05850325, 0.014532602, 0.0478...
Name: embedding, Length: 2645, dtype: object


In [ ]:
# Prepare embeddings as input for the Autoencoder
embeddings = np.array(log_data['embedding'].tolist())

# Autoencoder
input_dim = embeddings.shape[1]
encoding_dim = 64  # Size of the latent space
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Compile and train the Autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(embeddings, embeddings, epochs=100, batch_size=32, shuffle=True)

# Compute reconstruction error for each log
reconstructed = autoencoder.predict(embeddings)
reconstruction_error = np.mean((embeddings - reconstructed) ** 2, axis=1)

# Flag anomalies based on reconstruction error threshold
threshold = np.percentile(reconstruction_error, 95)  # 95th percentile
log_data['is_anomaly'] = reconstruction_error > threshold

In [36]:
log_data['@timestamp'] = pd.to_datetime(log_data['@timestamp'], errors='coerce')

if 'message' not in log_data.columns:
    raise KeyError("The 'message' column is not found in the dataset. Ensure it exists.")
else:
    print("The 'message' column is present and will be used.")

log_data = log_data[['message', '@timestamp', 'is_anomaly']]

# graph for event correlation
G = nx.Graph()

# Add nodes for log events
# Change: Add all nodes to the graph with their message and timestamp, and an anomaly flag
for index, row in log_data.iterrows():
    G.add_node(index, message=row['message'], timestamp=row['@timestamp'], anomaly=row['is_anomaly'])

# Add edges based on temporal and semantic similarity
for i in range(len(log_data)):
    for j in range(i + 1, len(log_data)):
        if pd.isnull(log_data.loc[i, '@timestamp']) or pd.isnull(log_data.loc[j, '@timestamp']):
            continue

        # Temporal proximity
        time_diff = abs((log_data.loc[i, '@timestamp'] - log_data.loc[j, '@timestamp']).total_seconds())
        if time_diff < 300:  # 5 minutes threshold
            # Cosine similarity
            sim = np.dot(embeddings[i], embeddings[j]) / (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
            if sim > 0.8:  # Similarity threshold
                G.add_edge(i, j, weight=sim)

# Use graph algorithms to find correlated clusters
clusters = list(nx.connected_components(G))
print(f"Detected {len(clusters)} clusters of anomalies!")

for cluster_id, cluster in enumerate(clusters):
    print(f"Cluster {cluster_id}:")
    for node in cluster:
        # Change: Access the message attribute from the node data
        print(f"  - Log: {G.nodes[node]['message']}")

<ipython-input-36-4ab95501f5cb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_data['@timestamp'] = pd.to_datetime(log_data['@timestamp'], errors='coerce')


The 'message' column is present and will be used.
Detected 78 clusters of anomalies!
Cluster 0:
  - Log: E0901 12:32:40.190175   16374 pod_workers.go:191] Error syncing pod 320b844c-b408-4354-a7ce-3f37a75f79ed ("aspen-mesh-packet-inspector-wdjzg_istio-system(320b844c-b408-4354-a7ce-3f37a75f79ed)"), skipping: failed to "StartContainer" for "aspen-mesh-packet-inspector" with CrashLoopBackOff: "back-off 5m0s restarting failed container=aspen-mesh-packet-inspector pod=aspen-mesh-packet-inspector-wdjzg_istio-system(320b844c-b408-4354-a7ce-3f37a75f79ed)"
  - Log: E0901 12:34:19.737803   13451 pod_workers.go:965] "Error syncing pod, skipping" err="failed to \"StartContainer\" for \"packet-inspector-aggregator\" with CrashLoopBackOff: \"back-off 5m0s restarting failed container=packet-inspector-aggregator pod=packet-inspector-aggregator-fqrlg_istio-system(19826f53-6c92-49dc-8d4b-56ec921fc4e3)\"" pod="istio-system/packet-inspector-aggregator-fqrlg" podUID=19826f53-6c92-49dc-8d4b-56ec921fc4e3
  

In [37]:
incidents = []
for cluster_id, cluster in enumerate(clusters):
    try:
        leading_edge = min(
            cluster,
            key=lambda node: G.nodes[node].get('timestamp', pd.Timestamp.max)
        )
        related_logs = [G.nodes[node]['message'] for node in cluster]

        incidents.append({
            "incident_id": cluster_id,
            "leading_edge": G.nodes[leading_edge]['message'],
            "timestamp": G.nodes[leading_edge]['timestamp'],
            "related_logs": related_logs
        })
    except Exception as e:
        print(f"Error processing cluster {cluster_id}: {e}")

incident_df = pd.DataFrame(incidents)
incident_df.to_csv('/content/incidents.csv', index=False)
print("Incident report saved!")

Incident report saved!
